In [7]:
import psycopg2
import os
import time
from dotenv import load_dotenv

# 1. 환경 변수 로드
env_path = r"..\server\.env"
if os.path.exists(env_path):
    load_dotenv(dotenv_path=env_path)
    print(f"✅ 환경 변수 로드 성공: {env_path}")

# 설정 경로
base_volume_path = r"..\server\volume"
save_path = r"..\qa\model_qa_results"


✅ 환경 변수 로드 성공: D:\final_project\server\.env
=== [QA] 데이터 무결성 검증 시작 (대상: analysis_materials) ===
총 4건의 레코드를 분석합니다...

>> 모든 DB 기록과 물리 파일이 일치합니다. (정상)

상세 검증 결과가 저장되었습니다: D:\final_project\qa\model_qa_results\integrity_check_report.txt


In [ ]:
def check_data_integrity_postgresql():
    db_info = {
        'dbname': os.getenv('DB_NAME'),
        'user': os.getenv('DB_USER'),
        'password': os.getenv('DB_PASSWORD'),
        'host': os.getenv('DB_HOST'),
        'port': os.getenv('DB_PORT')
    }

    try:
        conn = psycopg2.connect(**db_info)
        cur = conn.cursor()
    except Exception as e:
        print(f"❌ DB 연결 실패: {e}")
        return

    target_table = 'analysis_materials'
    print(f"=== [QA] 데이터 무결성 검증 시작 (대상: {target_table}) ===")

    verified_details = [] # 정상 매칭 내역 저장
    missing_files_list = [] # 유실 내역 저장
    rows = []

    try:
        # DB 데이터 조회 (이미지 및 엑셀 경로)
        cur.execute(f'SELECT id, defect_image_url, performance_data_url FROM "{target_table}"')
        rows = cur.fetchall()
        print(f"총 {len(rows)}건의 레코드를 분석합니다...\n")

        for row in rows:
            res_id, img_url, excel_url = row
            
            def get_full_info(db_path, sub_folder):
                if not db_path: return "N/A", "N/A", False
                filename = os.path.basename(db_path.replace('\\', '/'))
                full_path = os.path.join(base_volume_path, sub_folder, filename)
                exists = os.path.exists(full_path)
                return filename, full_path, exists

            # 각 파일별 정보 추출 및 검증
            img_name, img_path, img_ok = get_full_info(img_url, 'detected_results')
            excel_name, excel_path, excel_ok = get_full_info(excel_url, 'performance_data')

            status_text = "정상" if img_ok and excel_ok else "유실"
            
            detail_info = {
                'id': res_id,
                'img_name': img_name,
                'img_ok': img_ok,
                'excel_name': excel_name,
                'excel_ok': excel_ok,
                'status': status_text
            }

            if img_ok and excel_ok:
                verified_details.append(detail_info)
            else:
                missing_files_list.append(detail_info)
                print(f" [유실 발견] ID {res_id}: 이미지({img_ok}), 엑셀({excel_ok})")

        if not missing_files_list and len(rows) > 0:
            print(">> 모든 DB 기록과 물리 파일이 일치합니다. (정상)")

    except Exception as e:
        print(f"❌ 분석 오류: {e}")

    cur.close()
    conn.close()

    # --------------------------------------------------
    # 2. 결과 저장 로직 (상세 내역 포함)
    # --------------------------------------------------
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    file_name = os.path.join(save_path, "integrity_check_report.txt")
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write("==================================================\n")
        f.write("데이터 무결성 검증 상세 리포트\n")
        f.write(f"검사 시각: {time.strftime('%Y-%m-%d %H:%M:%S')}\n")
        f.write(f"검증 대상 테이블: {target_table}\n")
        f.write(f"기준 Root: {base_volume_path}\n")
        f.write("-" * 50 + "\n")
        
        f.write(f"[요약 결과]\n")
        f.write(f" - 전체 레코드: {len(rows)}건\n")
        f.write(f" - 정상 매칭: {len(verified_details)}건\n")
        f.write(f" - 유실 발생: {len(missing_files_list)}건\n")
        f.write("-" * 50 + "\n")

        if missing_files_list:
            f.write("[유실 데이터 내역]\n")
            for m in missing_files_list:
                f.write(f"ID {m['id']} | 이미지: {m['img_name']}({m['img_ok']}) | 엑셀: {m['excel_name']}({m['excel_ok']})\n")
            f.write("-" * 50 + "\n")

        f.write("[정상 매칭 상세 내역]\n")
        for v in verified_details:
            f.write(f"ID {v['id']} [매칭 완료]\n")
            f.write(f"  └ 이미지: {v['img_name']} (위치: detected_results/)\n")
            f.write(f"  └ 엑셀  : {v['excel_name']} (위치: performance_data/)\n")
            f.write("\n")
            
        f.write("==================================================\n")

    print(f"\n상세 검증 결과가 저장되었습니다: {file_name}")

In [ ]:
# 실행
check_data_integrity_postgresql()